In [ ]:
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import torch

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from fccd.data.modules import CDDDataModule
from fccd.data.datasets import TabularDataset
from fccd.models.LSTM import RepeatedInputLSTM, EncodedHiddenStateLSTM, ChunkingEncodedHiddenStateLSTM
from fccd.models.baseline import KNNBaseline
from fccd.models.GBT import LightGBMModel
from fccd.util import plot_prediction_vs_truth, plot_prediction_vs_truth_sklearn, limit_psd
from fccd.util import collect_dataloader

%load_ext autoreload
%autoreload 2

## Load and prepare data

First load and prepare the data. All other notebooks assume the data to be located in "$PROJ_ROOT/data/processed/".

We first load the data, then we apply the limit_psd step. If the PSD estiamte is higher than the actual highest stress level, this may cause errors. Hence we limit the PSDs to be lesser or equal to the highest stress level of the corresponding series.

In [ ]:
# Read data
cu_125 = pd.read_csv("../data/processed/cu_125.csv", index_col=0)
cu_1000 = pd.read_csv("../data/processed/cu_1000.csv", index_col=0)

al_125 = pd.read_csv("../data/processed/al_125.csv", index_col=0)
al_1000 = pd.read_csv("../data/processed/al_1000.csv", index_col=0)

au_125 = pd.read_csv("../data/processed/au_125.csv", index_col=0)
au_1000 = pd.read_csv("../data/processed/au_1000.csv", index_col=0)

data = pd.concat([cu_125])
#data_single = pd.concat([cu_125, cu_1000, al_125, al_1000, au_125, au_1000])   # Loading all data (Memory-heavy)

data = data.reset_index(drop=True)

# GPa to MPa, optional
data["stress"] = data["stress"] * 1000
data["psd"] = data["psd"] * 1000

In [ ]:
# Prepare Data
data = limit_psd(data)

## Creating Dataset

For creating a dataset we create LightningDatamodules called CDDDataModule. The modules and its underlying datasets handle the scaling and preprocessing

For Mulit-material we can just use multiple datasets, for multi-target we need to specify the multi-target parameter.

### Creating Single target datasets

data:           the dataframe
target:         name of the target in df
psd:            name of psd in df
t:              name of time index in df
group:          name of series identifier in df

drop_cols:      get dropped during preprocessing, exlcude obselete columns and different targets

transform:      provide a sklearn scaler if scaling is desired

split_dataset:  true if computation of split is needed, (Use true if using the LSTM+PDP model, False otherwise)

In [ ]:
# Create a single-target dataset

stress_dm = CDDDataModule(
    data,
    target="dislocation",
    psd="psd",
    group="id",
    drop_cols=["strain", "time_ns", "stress"],
    time="t",
    batch_size=64,
    categoricals=["material", "euler_angles", "mesh"],
    num_dataloader_workers=8,
    transform=MinMaxScaler,
    split_dataset=True,
)
stress_dm.setup()

In [ ]:
# Create a multi-target dataset
# The target does not matter actually

stress_dm = CDDDataModule(
    data,
    psd="psd",
    target="all",
    multi_target=True,
    group="id",
    drop_cols=["time_ns"],
    time="t",
    batch_size=64,
    categoricals=["material", "euler_angles", "mesh"],
    num_dataloader_workers=0,
    transform=MinMaxScaler,
    split_dataset=True,
)
stress_dm.setup()

## Training the models

Training the models stays the same for all three single/multi-material and single/multi-target, we just need to provide the proper datamodule.




### Sklearn models

Training models following the sklaern API.


In [ ]:
def train_sklearn_model(model, datamodule):
    train_dataloader = datamodule.train_dataloader()
    val_dataloader = datamodule.train_dataloader()

    X_train, y_train = collect_dataloader(train_dataloader)
    X_val, y_val = collect_dataloader(val_dataloader)
    model.fit(X_train, y_train)

    test_dataloader = datamodule.test_dataloader()
    X_test, y_test = collect_dataloader(test_dataloader)
    y_test_pred = model.predict(X_test)

    test_loss = mean_squared_error(y_test, y_test_pred)
    return test_loss

In [ ]:
model.predict([[0.01, 0.2, 0.9]])

In [ ]:
knn = KNNBaseline(1)
train_sklearn_model(knn, datamodule=stress_dm)

In [ ]:
gbm = LightGBMModel()
train_sklearn_model(gbm, stress_dm)

In [ ]:
plot_prediction_vs_truth_sklearn(stress_dm.test_dataloader(), {"gbm": gbm, "knn_bs": knn}, 2)

### Repeated Input LSTM

This LSTM takes all parameters at every time step as input plus past predictions resulting in a structure like
with $x_i$ and i the ith

($x_{1}$ $x_{2}$ $x_{3}$ $0$) -> ($x_{1}$ $x_{2}$ $x_{3}$ $y_0$) -> ... -> ($x_{1}$ $x_{2}$ $x_{3}$ $y_{t-1}$)

In [ ]:
repin = RepeatedInputLSTM(n_hidden=30, input_size=20)

# For multi-target just specify the corresponding output size
repin = RepeatedInputLSTM(n_hidden=30, input_size=20, output_size=3)

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=3e-10, patience=10, verbose=False, mode="min")

trainer = pl.Trainer(
    max_epochs=10,                      # Max  training epochs: Hard limit
    accelerator="gpu",                  # cpu/gpu/mps Use cuda gpu for fast training
    callbacks=[early_stop_callback],    # Early stopping if accuracy does not increase on validation set (confifgure delta and patience above)
    auto_lr_find=True                   # Automatically find learning rate for LSTM, called in trainer.tune()
)
trainer.tune(repin, datamodule=stress_dm)
trainer.fit(repin, datamodule=stress_dm)
#trainer.test(repin, datamodule=stress_dm)

In [ ]:
# Plot predictions of model vs ground truth
plot_prediction_vs_truth(stress_dm.test_dataloader(), {"repin": repin}, 2, 400)

### Encoded Hidden State LSTM

LSTM with initial sequence input 0 that uses encoding of input parameters as initial hidden and cell state.

In [ ]:
encoded_hidden_lstm = EncodedHiddenStateLSTM(input_size=20, lstm_layers=1, lstm_hidden_size=30, mlp_hidden_layers=2, output_size=1)

# For multi target just specify to proper output size
encoded_hidden_lstm_multi_target = EncodedHiddenStateLSTM(input_size=20, lstm_layers=1, lstm_hidden_size=30, mlp_hidden_layers=2, output_size=3)


early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=3e-6, patience=100, verbose=False, mode="min")

trainer = pl.Trainer(
    max_epochs=1500, 
    accelerator="gpu",
    callbacks=[early_stop_callback],
    auto_lr_find=True
)
trainer.tune(encoded_hidden_lstm, datamodule=stress_dm)
trainer.fit(encoded_hidden_lstm, datamodule=stress_dm)
#trainer.test(encoded_decoded, datamodule=stress_dm)

In [ ]:
plot_prediction_vs_truth(stress_dm.train_dataloader(), {"encoded_decoded": encoded_hidden_lstm}, 2, 400)

### Training LSTM+PDP

Training the LSTM+PDP Hybrid. First we need to create a dataset to train the PDP-index estimate, then we create a model and pass a wrapper method that provides estimates.

In [ ]:
train_data = stress_dm.train_dataset()
val_data = stress_dm.val_dataset()
test_data = stress_dm.test_dataset()

# Create dataset to train PDP index estimator
psd_train_data = TabularDataset.psd_from_cdd_dataset(train_data)
psd_val_data = TabularDataset.psd_from_cdd_dataset(val_data)
psd_test_data = TabularDataset.psd_from_cdd_dataset(test_data)

x_train, y_train = psd_train_data.dataset
x_val, y_val = psd_val_data.dataset
x_test, y_test = psd_test_data.dataset

stackedx_train = torch.cat([x_train, x_val], dim=0)
stackedy_train = torch.cat([y_train, y_val], dim=0)

In [ ]:
# Create and fit estimator
rf = RandomForestRegressor(n_estimators=1000)
rf.fit(stackedx_train, stackedy_train)

In [ ]:
# Wrapper method to provide PDP index estimates
def estimate_rounded_rf(static_parameters, psd_data, device):
    regression_input = torch.hstack([static_parameters, psd_data])
    regression_input = regression_input.detach().clone()
    
    if regression_input.device != "cpu":
        regression_input = regression_input.cpu()

    prediction = rf.predict(regression_input)
    result = torch.from_numpy(prediction)
    result = result.to(device)
    return result

In [ ]:
chenc = ChunkingEncodedHiddenStateLSTM(20, 30, 5, 1).with_estimator(estimate_rounded_rf)

# For multi-target specify the outuput size and the index of the target that corresponds to stress. The model predicts stress using the PDP estimator + the LSTM
chenc_multi = ChunkingEncodedHiddenStateLSTM(20, 30, 5, 1, output_size=3, target_stress_idx=0).with_estimator(estimate_rounded_rf)

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=3e-10, patience=100, verbose=False, mode="min")

trainer = pl.Trainer(
    max_epochs=1500,
    accelerator="gpu",
    callbacks=[early_stop_callback],
    auto_lr_find=True,
)

In [ ]:
#trainer.tune(chenc, datamodule=stress_dm)
trainer.fit(chenc, datamodule=stress_dm)

In [ ]:
plot_prediction_vs_truth(stress_dm.train_dataloader(), {"chenc": chenc}, 5, 400)